# Project

In [1]:
import numpy as np
import math
import time
from computer_vision import *
from dijkstra import compute_shortest_path
from kalman import kalman_ext_filter
import tdmclient.notebook
await tdmclient.notebook.start()


In [2]:
# parametres
speed_r = 100
speed_f = 100
stop = 0
MIN_DIST = 40
ANGLE_TOLERANCE = 0.30
PERIOD = 0.001
SPEED_AVG = 150
ROBOT_SPEED_TO_MM = 140/500

@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    global motor_left_target, motor_right_target
    
    motor_left_target = l_speed
    motor_right_target = r_speed

def get_angle_between(vec1, vec2):
    vec1_unit = vec1 / np.linalg.norm(vec1)
    vec2_unit = vec2 / np.linalg.norm(vec2)

    return np.arccos(np.dot(vec1_unit, vec2_unit))

def distance (x1, y1, x2, y2):
    dist = np.sqrt((x1-x2)**2 + (y1-y2)**2)
    return dist

def wrap_angle(angle):
    if angle > math.pi:
        angle = angle - 2*math.pi
    elif angle < -math.pi:
        angle = angle + 2*math.pi
    return angle

In [3]:
def controller(goal_dist, angle):
    kp_rot = 100
    kp_lin = 3

    if abs(angle) > ANGLE_TOLERANCE:
        speed_l = 80 - kp_rot*(angle)
        speed_r = 80 + kp_rot*(angle)

    else:
        speed_l = SPEED_AVG
        speed_r = SPEED_AVG
    return speed_l, speed_r

In [4]:
def pathing (video_capture, filter, path, x0, y0, theta0, periode=0.01):
    camera_hist = []
    kalman_hist = []
    error_hist = []
    teta = np.zeros(np.shape(path)[0])
    speed_l = speed_r = 0
    goalx = path[0][0]
    goaly = path[0][1]
    
    i = 0
    x_pense = x0
    y_pense = y0
    curr_time = 0
    
    kx = kteta = ky = angle = 0
    robot_detected = False
    camera_available = False
    dist = distance (goalx, goaly, kx, ky)
    while True:       
        #   step 1: get the image, detect the robot, and get the position
        camera_available, frame = video_capture.read()
        position_measure = []
        if camera_available == True:
            position_measure, position_contour, proccessed_framed = computer_vision(frame, 'robot', False)
            # cv has detected the robot, we can use the position
            if len(position_measure) == 1:
                robot_detected = True
                position_measure = position_measure[0]

                if position_measure is not None:                    
                    _, theta_measure = get_robot_position(frame, position_measure, position_contour)
                    
                    position_measure = invert_coordinates(position_measure)
                    position_measure[0:2] = pixel_to_metric(position_measure[0:2])

                    [kx, ky, kteta] = [position_measure[0], position_measure[1], -theta_measure]
                    measurement = np.array([kx, ky, kteta])
                else:
                    robot_detected = False
            else:
            # robot not detected, we use kalman
                robot_detected = False

        if camera_available == False or robot_detected == False:
            measurement = np.array([0,0,0])         # CHECK WHY 0,0,0            
        
        [pred_x, pred_y, pred_teta], _, curr_time = filter.filter(measurement, curr_time) 
        
        dist = distance (goalx, goaly, pred_x, pred_y)
        teta[i] = math.atan2((goaly - pred_y), (goalx - pred_x)) #change angle according to kalman
    

        if (dist < MIN_DIST):
            if i < np.shape(path)[0] - 1 :
                i = i + 1
                goalx = path[i][0]
                goaly = path[i][1]
                x_pense = path[i-1][0]
                y_pense = path[i-1][1]
                vec1 = [np.cos(teta[i-1]), np.sin(teta[i-1])]
                vec2 = [goalx - x_pense, goaly - y_pense]
                angle = get_angle_between(vec1, vec2)
            else:
                motors(stop, stop)
                break
        else :
            x_pense = pred_x + speed_f * periode * np.cos(teta[i]) #à update
            y_pense = pred_y + speed_f * periode * np.sin(teta[i])
            angle =  wrap_angle((teta[i] - pred_teta) % (2*np.pi))
            
            #speed_l, speed_r = controller(dist, angle)
            speed_l  = SPEED_AVG
            speed_r = SPEED_AVG
        
        # set input for kalman filter
        filter.u = np.array([speed_l, speed_r]) * ROBOT_SPEED_TO_MM

        motors(int(speed_l),int(speed_r))

        controller(dist,angle)
        #print(i, camera_available, dist, position_measure, angle, teta[i], kteta)
        
        print("camera : %.3f %.3f %.4f" %(kx, ky, kteta))
        print("kalman : %.3f %.3f %.4f" %(pred_x, pred_y, pred_teta))
        print("error : %.3f %.3f %.4f" %(kx-pred_x, ky-pred_y, kteta-pred_teta))
        camera_hist = np.append(camera_hist, [kx, ky, kteta])
        kalman_hist = np.append(kalman_hist, [pred_x, pred_y, pred_teta])
        error_hist = np.append(error_hist, [kx-pred_x, ky-pred_y, kteta-pred_teta])

        #cv2.imshow("frame", cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    motors(stop, stop)
    return camera_hist, kalman_hist, error_hist

# Main 

1
2
3
start = [151.184 694.931] goal = [1025.646  584.12 ]


In [5]:
video_capture = cv2.VideoCapture(0,cv2.CAP_DSHOW)
fig, ax = plt.subplots(figsize=(8,8))
cv_successful, obst, robot, goal, frame = cv_start(video_capture, show_image= True, exposure=-7)

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
obst = format_contour(obst)

if cv_successful:
    start = np.array([robot[0], robot[1]])
    goal = np.array([goal[0], goal[1]])
    path = compute_shortest_path(obst, start, goal)
    path = np.rint(path).astype(int)
    frame = draw_path(frame, path) 
        
    metric_path = np.zeros(np.shape(path))

    # invert y axis
    for i in range(len(path)):
        path[i] = invert_coordinates(path[i])
    metric_start = invert_coordinates(start)
    metric_goal = invert_coordinates(goal)
    # change to metric  

    for i in range(len(path)):
        metric_path[i] = pixel_to_metric(path[i])
    metric_start = pixel_to_metric(metric_start)
    metric_goal = pixel_to_metric(metric_goal)
    
    filter = kalman_ext_filter(metric_start[0], metric_start[1], -robot[2], 0, 0 ,SPEED_AVG, PERIOD)
    print('start =', metric_start, 'goal =', metric_goal)

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [8]:
#ret, frame = video_capture.read()
#print(ret)
#plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
#cv2.imwrite('frame.jpeg', frame)

motors(stop, stop)
np.shape(path)
print(np.shape(path)[0])

11


In [7]:
cam, kal, err =  pathing(video_capture, filter, metric_path, metric_start[0], metric_start[1], -robot[2], periode=PERIOD)
print("finished")
motors(stop, stop)


#plot the results
fig, ax = plt.subplots(figsize=(8,8))
ax.plot(cam[0::3], cam[1::3], label='camera')
ax.plot(kal[0::3], kal[1::3], label='kalman')
ax.plot(err[0::3], err[1::3], label='error')
ax.legend()
plt.show()


camera : 151.184 694.931 0.0341
kalman : 151.184 694.931 0.0341
error : 0.000 0.000 0.0000
camera : 151.184 694.931 0.0225
kalman : 151.184 694.931 0.0283
error : 0.000 0.000 -0.0058
camera : 151.184 694.931 0.0225
kalman : 154.267 695.035 0.0262
error : -3.083 -0.104 -0.0038
camera : 151.184 694.931 0.0225
kalman : 156.804 695.120 0.0252
error : -5.620 -0.189 -0.0027
camera : 152.043 694.931 0.0112
kalman : 159.219 695.196 0.0222
error : -7.176 -0.265 -0.0110
camera : 160.633 692.354 -0.0681
kalman : 163.241 694.850 0.0070
error : -2.608 -2.496 -0.0751
camera : 170.082 689.777 -0.1500
kalman : 168.463 694.268 -0.0155
error : 1.619 -4.491 -0.1345
camera : 173.518 687.200 -0.2040
kalman : 173.294 693.526 -0.0392
error : 0.224 -6.326 -0.1648
camera : 178.672 685.482 -0.2533
kalman : 181.539 692.890 -0.0630
error : -2.867 -7.408 -0.1903
camera : 182.967 683.764 -0.3002
kalman : 189.012 692.225 -0.0867
error : -6.045 -8.461 -0.2135
camera : 189.839 680.328 -0.3036
kalman : 196.374 691.389 

KeyboardInterrupt: 